In [1]:
!pip install -e ../TrackSelectorDNN/ &> /tmp/install.log

In [9]:
from cluster.RayJobManager import RayJobManager

In [10]:
RAY_ADDRESS  = "http://10.254.168.230:8265"
#RAY_ADDRESS  = "local"
ENTRYPOINT   = "python3 cluster/tune_grid.py"
WORKING_DIR  = "/eos/user/e/ecoradin/GitHub/TrackSelectorDNN-Ray/"
PIP = {
    "packages": [
        "torch",
        "ray[tune]",
        "git+https://github.com/EmanueleCoradin/TrackSelectorDNN.git@main"
    ],
    "pip_check": False,
    "pip_install_options": ["--verbose"]
}

ray_manager = RayJobManager(ray_address=RAY_ADDRESS)

In [11]:
job_id = ray_manager.submit_job(
    entrypoint=ENTRYPOINT,
    runtime_env={
        "working_dir": WORKING_DIR, 
        "pip": PIP, 
    }
    
)

ray_manager.wait_until_done(job_id)

2025-11-12 20:56:00,373	INFO dashboard_sdk.py:338 -- Uploading package gcs://_ray_pkg_9affebc20d5d1549.zip.
2025-11-12 20:56:00,374	INFO packaging.py:588 -- Creating a file package for local module '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN-Ray/'.


Submitted Ray job: raysubmit_NbrgfhQYHqkd7ZaL
Job status: PENDING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: SUCCEEDED

--- Job Logs ---
2025-11-12 12:56:01,111	INFO job_manager.py:568 -- Runtime env is setting up.
/home/ray/anaconda3/lib/python3.9/site-packages/ray/tune/impl/tuner_internal.py:144: RayDeprecationWarning: The `RunConfig` class should be imported from `ray.tune` when passing it to the Tuner. Please update your imports. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the environment variable: RAY_TRAIN_ENABLE_V2_MIGRATION_WARNINGS=0
  _log_deprecation_warning(
2025-11-12 12:56:03,470	INFO worker.py:1692 -- Using address 10.100.192.236:6379 set in the environm

In [2]:
#Uncomment to run locally

from ray import tune
from TrackSelectorDNN.configs.schema import load_config
from TrackSelectorDNN.tune.trainable import trainable

base_cfg = load_config("base.yaml").model_dump()

search_space = {
    **base_cfg,  # include all validated defaults
    "lr": tune.loguniform(1e-5, 1e-2),
    "weight_decay": tune.loguniform(1e-5, 1e-2),
    "latent_dim": tune.choice([32, 64, 128]),
    "netA_hidden_dim": tune.choice([32, 64, 128]),
    "netB_hidden_dim": tune.choice([32, 64, 128]),
    "pooling_type": tune.choice(["sum", "mean", "softmax"]),
    "netA_hidden_layers": tune.choice([1, 2, 3, 4, 5]),
    "netB_hidden_layers": tune.choice([1, 2, 3, 4, 5]),
}


tuner = tune.Tuner(
    tune.with_resources(
        trainable,
        {"cpu": 1, "gpu": 0}  # adjust CPU/GPU numbers as needed
    ),
    param_space=search_space,
    tune_config=tune.TuneConfig(metric="val_loss", mode="min", num_samples=4),
)

results = tuner.fit()


(trainable pid=8495) [Epoch 1/5] train_loss=0.6964, val_loss=0.6352, val_acc=0.7177


(trainable pid=8495) /eos/user/e/ecoradin/.local/lib/python3.11/site-packages/ray/train/_internal/session.py:792: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(trainable pid=8495)   _log_deprecation_warning(
(trainable pid=8495) /eos/user/e/ecoradin/.local/lib/python3.11/site-packages/ray/tune/trainable/trainable_fn_utils.py:41: RayDeprecationWarning: The `Checkpoint` class should be imported from `ray.tune` when passing it to `ray.tune.report` in a Tune function. Please update your imports. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the environment variable: RAY_TRAIN_ENABLE_V2_MIGRATION_WARNINGS=0
(trainable pid=8495)   _log_deprecation_warning(
(trainable pid=8495) Checkpoint successfully cr

(trainable pid=8493) [Epoch 1/5] train_loss=0.2888, val_loss=0.2101, val_acc=0.9135 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


(trainable pid=8493) /eos/user/e/ecoradin/.local/lib/python3.11/site-packages/ray/train/_internal/session.py:792: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454 [repeated 2x across cluster]
(trainable pid=8493)   _log_deprecation_warning( [repeated 4x across cluster]
(trainable pid=8493) /eos/user/e/ecoradin/.local/lib/python3.11/site-packages/ray/tune/trainable/trainable_fn_utils.py:41: RayDeprecationWarning: The `Checkpoint` class should be imported from `ray.tune` when passing it to `ray.tune.report` in a Tune function. Please update your imports. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the environment variable: RAY_TRAIN_ENABLE_V2_MIGRATION_WARNINGS=0 [repeated 2x across cluster]
(trainabl

(trainable pid=8495) [Epoch 3/5] train_loss=0.5387, val_loss=0.5047, val_acc=0.8537 [repeated 2x across cluster]


...

(trainable pid=8492) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/eos/user/e/ecoradin/ray_results/trainable_2025-11-13_14-56-45/trainable_03c88_00003_3_latent_dim=32,lr=0.0020,netA_hidden_dim=128,netA_hidden_layers=1,netB_hidden_dim=32,netB_hidden_layers=3,p_2025-11-13_14-57-04/checkpoint_000003)


(trainable pid=8494) [Epoch 2/5] train_loss=0.1771, val_loss=0.1634, val_acc=0.9301


(trainable pid=8494) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/eos/user/e/ecoradin/ray_results/trainable_2025-11-13_14-56-45/trainable_03c88_00002_2_latent_dim=32,lr=0.0031,netA_hidden_dim=128,netA_hidden_layers=5,netB_hidden_dim=32,netB_hidden_layers=3,p_2025-11-13_14-57-04/checkpoint_000001)


(trainable pid=8494) [Epoch 3/5] train_loss=0.1736, val_loss=0.1644, val_acc=0.9289
(trainable pid=8494) [Epoch 4/5] train_loss=0.1701, val_loss=0.1662, val_acc=0.9279
(trainable pid=8494) [Epoch 5/5] train_loss=0.1687, val_loss=0.1602, val_acc=0.9319
(trainable pid=8494) [DEBUG] Final report to Ray: /eos/user/e/ecoradin/GitHub/TrackSelectorDNN/runs/03c88_00002


(trainable pid=8494) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/eos/user/e/ecoradin/ray_results/trainable_2025-11-13_14-56-45/trainable_03c88_00002_2_latent_dim=32,lr=0.0031,netA_hidden_dim=128,netA_hidden_layers=5,netB_hidden_dim=32,netB_hidden_layers=3,p_2025-11-13_14-57-04/checkpoint_000002)
(trainable pid=8494) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/eos/user/e/ecoradin/ray_results/trainable_2025-11-13_14-56-45/trainable_03c88_00002_2_latent_dim=32,lr=0.0031,netA_hidden_dim=128,netA_hidden_layers=5,netB_hidden_dim=32,netB_hidden_layers=3,p_2025-11-13_14-57-04/checkpoint_000003)
2025-11-13 15:24:52,457	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/eos/user/e/ecoradin/ray_results/trainable_2025-11-13_14-56-45' in 0.0975s.
2025-11-13 15:24:52,474	INFO tune.py:1041 -- Total run time: 1669.07 seconds (1668.50 seconds for the tuning loop).


In [3]:
best_result = results.get_best_result(metric="val_loss", mode="min")

print("Best config found:")
print(best_result.config)

best_ckpt_dir = best_result.checkpoint
print("Best checkpoint path:", best_ckpt_dir)

Best config found:
{'hit_input_dim': 9, 'track_feat_dim': 23, 'latent_dim': 32, 'pooling_type': 'softmax', 'netA_hidden_dim': 128, 'netA_hidden_layers': 5, 'netA_batchnorm': True, 'netA_activation': 'silu', 'netB_hidden_dim': 32, 'netB_hidden_layers': 3, 'netB_batchnorm': True, 'netB_activation': 'silu', 'lr': 0.003079479433839436, 'epochs': 5, 'batch_size': 128, 'patience': 15, 'delta': 0.0, 'dataset_type': 'production', 'dummy_load_path': '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/TrackSelectorDNN/data/dummy_dataset.pt', 'train_path': '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/TrackSelectorDNN/data/tracks_train_dataset.pt', 'val_path': '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/TrackSelectorDNN/data/tracks_train_dataset.pt', 'test_path': '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/TrackSelectorDNN/data/tracks_test_dataset.pt', 'n_tracks': 100000, 'max_hits': 16, 'val_fraction': 0.2, 'weight_decay': 0.00035202472362376376}
Best checkpoint path: Checkpoint(filesystem=local, 